# get(url)

Some great [Stata-to-Python](https://www.danielmsullivan.com/pages/tutorial_stata_to_python.html) code


Wednesday May 17,  12:00AM
```
(base) d@Poseidon 2.pantheon % streamlit run capstone.py

  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://10.0.0.250:8501

```

In [ ]:
# Abimereki Muzaale

import streamlit as st
import pandas as pd
import numpy as np
import pickle

# Load the model and beta coefficients
with open('model.pkl', 'rb') as file:
    model = pickle.load(file)
    
with open('betas.pkl', 'rb') as file:
    betas = pickle.load(file)

# Define a function to calculate the risk of mortality
def calculate_risk(demographic, history, exam, labs):
    # Combine all input features into a single array
    features = np.concatenate([demographic, history, exam, labs])
    
    # Calculate the log hazard ratio using the beta coefficients
    log_hr = np.sum(features * betas)
    
    # Calculate the absolute risk using the Cox proportional hazards formula
    risk = np.exp(log_hr) * baseline_risk
    
    return risk

# Define the baseline risk for a healthy individual
baseline_risk = 0.0028 #from literature & base-case survival

# Define the input fields for the calculator
age = st.slider('Age', 18, 100, 5)
female = st.selectbox('Sex', ['Male', 'Female'])
racecat = st.selectbox('Race', ['Hispanic', 'Other', 'White'])
dm = st.checkbox('Diabetes')
htn = st.checkbox('Hypertension')
acr = st.slider('ACR', 0.0, 100.0, 0.0)
hba1c = st.slider('HbA1c', 0.0, 20.0, 5.0)
egfr = st.slider('eGFR', 0, 200, 100)
sbp = st.slider('SBP', 50, 250, 120)

# Convert categorical variables to one-hot encoding
if female == 'Male':
    sex_male = 1
else:
    sex_male = 0
    
if racecat == 'Hispanic':
    race_hispanic = 1
    race_other = 0
    race_white = 0
elif racecat == 'Other':
    race_hispanic = 0
    race_other = 1
    race_white = 0
else:
    race_hispanic = 0
    race_other = 0
    race_white = 1

# Combine all input features into a single array
demographic = np.array([age, sex_male, race_hispanic, race_other, race_white])
history = np.array([int(dm), int(htn)])
exam = np.array([sbp])
labs = np.array([acr,hba1c, egfr])  # Add any additional lab features here if needed

# Calculate the risk of mortality using the input features
risk = calculate_risk(demographic, history, exam, labs)

# Display the calculated risk
st.write('30-Year Risk of Mortality:', risk)


# ds4ph Capstone

This app represents proof-of-concept for something much [bigger](https://grantome.com/grant/NIH/K08-AG065520-01):

## Perioperative and long-term risks following nephrectomy in older live kidney donors

### Abimereki Muzaale, MD MPH & PhD Candidate (Clinical Investigation)

The number of older adults (aged<55) undergoing live donor nephrectomy has quadrupled over the last two decades and this trend is likely to continue. However, a knowledge gap exists in our understanding of the resiliencies to physical stress in older donors. Beyond age-related vulnerability to surgical stress, this aging population may have significant glomerular senescence, an inadequate adaptive response to the 50% loss of nephron mass and, as such, a very high likelihood of CKD in their lifetime. Approximately 80% of older donors have a postdonation GFR<60 ml/min/1.73m2 and meet the standard definition of CKD. 

As older donors progress towards CKD, they are likely to become frail and, as such, at increased risk of hospitalization. No study has focused on the identification of clinical signatures of resiliency in older persons undergoing live donor nephrectomy. The renal system is directly impacted by nephrectomy and its resiliency may affect the health and well-being of older donors. Provider discussions with these potential donors should focus on the patient's most important question: **Can I safely donate my kidney??** not the population-level question asked by the 2017 KDIGO guidelines: How many ESRD events in younger donors may be averted by transplanting older donors instead. I propose the following scientific aims: 

1. To quantify donation-attributable risk of ESRD and mortality in older donors; 
2. To describe the prevalence of hospitalizations in older donors vs. older nondonors; and,
3. To <u>implement a risk calculator</u> for ESRD, **mortality** and sentinel hospitalization in older donors. In brief, I'm in the third year of a PhD in Clinical Investigation Program at Bloomberg School. My efforts in this class are culminating in the creation of online risk calculators that inform the conversation between candidates and providers about the propriety of donation. I will deploy several other calculators to reflect my other outcomes of importance to donors other than mortality.

[Here](https://jhjhm.zoom.us/rec/share/fMFi8MmzKWtWgmdwEFgTgvwjEjQIT2JkJXNyTRvdSgY60uWGxSN8VPMja6wJafDK.zsVmahonhaML1VoY?startTime=1684297376000) is a brief video describing the webApp. Passcode: =67zSM#w

[![Review Assignment Due Date](https://classroom.github.com/assets/deadline-readme-button-24ddc0f5d75046c5622901739e7c5dd533143b0c8e959d652212380cedb1ea36.svg)](https://classroom.github.com/a/V-Ayz7c2)
